<a href="https://colab.research.google.com/github/rufous86/hh_api/blob/main/hh_api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install geopy

In [1]:
import requests
from tqdm.notebook import tqdm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('seaborn')
import plotly.express as px

In [2]:
res = []
def get_vacancy(vacancies, pages=50):
    for vacancy in vacancies:
        print(f'collecting {vacancy}')
        for page in tqdm(range(pages)):
            params = {
                'text': f'NAME:{vacancy}',
                'page': page,
                'per_page': 100
                }
            req = requests.get('https://api.hh.ru/vacancies', params).json()
            if 'items' in req.keys():
                res.extend(req['items'])

vacancies = ['аналитик', 'данные', 'data', 'analyst']
get_vacancy(vacancies)

data = pd.DataFrame(res)
data.shape

collecting аналитик


  0%|          | 0/50 [00:00<?, ?it/s]

collecting данные


  0%|          | 0/50 [00:00<?, ?it/s]

collecting data


  0%|          | 0/50 [00:00<?, ?it/s]

collecting analyst


  0%|          | 0/50 [00:00<?, ?it/s]

(5079, 30)

In [65]:
df = data.copy()
columns = ['name', 'salary', 'schedule', 'area']
df = df[columns]
df.head()

,name,salary,schedule,area
0,Аналитик,"{'from': 700000, 'to': 900000, 'currency': 'KZ...","{'id': 'fullDay', 'name': 'Полный день'}","{'id': '160', 'name': 'Алматы', 'url': 'https:..."
1,Системный аналитик\Бизнес аналитик,"{'from': 150000, 'to': 250000, 'currency': 'RU...","{'id': 'fullDay', 'name': 'Полный день'}","{'id': '2', 'name': 'Санкт-Петербург', 'url': ..."
2,Аналитик,"{'from': 70000, 'to': 70000, 'currency': 'RUR'...","{'id': 'fullDay', 'name': 'Полный день'}","{'id': '26', 'name': 'Воронеж', 'url': 'https:..."
3,Аналитик,"{'from': 80000, 'to': None, 'currency': 'RUR',...","{'id': 'fullDay', 'name': 'Полный день'}","{'id': '1', 'name': 'Москва', 'url': 'https://..."
4,Бизнес-аналитик,"{'from': 130000, 'to': 130000, 'currency': 'RU...","{'id': 'fullDay', 'name': 'Полный день'}","{'id': '2', 'name': 'Санкт-Петербург', 'url': ..."


In [66]:
df["city"] = (df["area"]
                     .apply(lambda x: x.get("name") 
                                      if isinstance(x, dict) 
                                      else np.nan))
df["schedule"] = (df["schedule"]
                     .apply(lambda x: x.get("id") 
                                      if isinstance(x, dict) 
                                      else np.nan))
df["salary_from"] = (df["salary"]
                     .apply(lambda x: x.get("from") 
                                      if isinstance(x, dict) 
                                      else np.nan))
df["salary_to"] = (df["salary"]
                     .apply(lambda x: x.get("to") 
                                      if isinstance(x, dict) 
                                      else np.nan))
df["salary_currency"] = (df["salary"]
                     .apply(lambda x: x.get("currency") 
                                      if isinstance(x, dict) 
                                      else np.nan))
# df["lat"] = (df["address"]
#                      .apply(lambda x: x.get("lat") 
#                                       if isinstance(x, dict) 
#                                       else np.nan))
# df["lon"] = (df["address"]
#                      .apply(lambda x: x.get("lng") 
#                                       if isinstance(x, dict) 
#                                       else np.nan))

df = df.drop(['salary', 'area'], axis=1)

In [104]:
df.head()

,name,schedule,city,salary_from,salary_to,salary_currency
0,Аналитик,fullDay,Алматы,700000.0,900000.0,KZT
1,Системный аналитик\Бизнес аналитик,fullDay,Санкт-Петербург,150000.0,250000.0,RUR
2,Аналитик,fullDay,Воронеж,70000.0,70000.0,RUR
3,Аналитик,fullDay,Москва,80000.0,NaN,RUR
4,Бизнес-аналитик,fullDay,Санкт-Петербург,130000.0,130000.0,RUR


In [105]:
df['salary_currency'].value_counts()

RUR    1268
USD      52
KZT      39
UZS      13
EUR      11
BYR      10
KGS       1
AZN       1
Name: salary_currency, dtype: int64

In [106]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5079 entries, 0 to 5078
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             5079 non-null   object 
 1   schedule         5079 non-null   object 
 2   city             5079 non-null   object 
 3   salary_from      1190 non-null   float64
 4   salary_to        812 non-null    float64
 5   salary_currency  1395 non-null   object 
dtypes: float64(2), object(4)
memory usage: 238.2+ KB


In [58]:
# df = df[df['salary_currency'] == 'RUR'][['name', 'schedule', 'city', 'salary_from', 'salary_to', 'salary_currency', 'lat', 'lon']]

In [59]:
# df = df.dropna(axis=0, subset=['lat', 'lon'])
# df.info()

In [123]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="geo")

def get_coords(city):
    geo = geolocator.geocode(city)
    if geo:
        return geo.longitude, geo.latitude
    else:
        return np.nan, np.nan


coords = pd.DataFrame(data=df['city'].unique(), columns=['city'])
coords['(lon, lat)'] = (coords['city']
                        .apply(lambda x: get_coords(x)))


coords['lon'] = (coords['(lon, lat)']
                        .apply(lambda x: x[0]))
coords['lat'] = (coords['(lon, lat)']
                        .apply(lambda x: x[1]))
coords = coords.drop('(lon, lat)', axis=1)

In [134]:
total_vac = dict(df
            .groupby('city')['name']
            .agg('count')
            )
total_vac

{'Алматы': 146,
 'Альметьевск': 1,
 'Анапа': 1,
 'Апрелевка': 3,
 'Армения': 9,
 'Архангельск': 1,
 'Астана': 33,
 'Астрахань': 3,
 'Атырау': 9,
 'Баку': 2,
 'Барнаул': 10,
 'Батуми': 5,
 'Белгород': 9,
 'Бердск': 1,
 'Бийск': 1,
 'Бишкек': 13,
 'Благовещенск (Амурская область)': 3,
 'Брест': 1,
 'Брянск': 8,
 'Великий Новгород': 2,
 'Видное': 1,
 'Витебск': 1,
 'Владивосток': 32,
 'Владимир': 18,
 'Волгоград': 6,
 'Волжский (Волгоградская область)': 1,
 'Вологда': 5,
 'Воронеж': 47,
 'Воскресенск': 2,
 'Всеволожск': 1,
 'Гродно': 1,
 'Дзержинск (Нижегородская область)': 2,
 'Домодедово': 1,
 'Дубна': 1,
 'Екатеринбург': 117,
 'Елабуга': 1,
 'Железногорск (Курская область)': 1,
 'Железнодорожный': 2,
 'Зеленогорск (Ленинградская обл)': 1,
 'Зеленоград': 5,
 'Зеленодольск (Республика Татарстан)': 1,
 'Иваново (Ивановская область)': 7,
 'Ижевск': 19,
 'Иннополис': 3,
 'Иркутск': 8,
 'Истра': 1,
 'Йошкар-Ола': 3,
 'Казань': 89,
 'Калининград': 5,
 'Калуга': 1,
 'Каменск-Уральский': 1,
 'К

In [135]:
df = df.merge(coords, on='city')
df['total_vac'] = (df['city']
                .apply(lambda x: total_vac[x]))
df.head()

,name,schedule,city,salary_from,salary_to,salary_currency,lon_x,lat_x,total_vac,lon_y,lat_y,lon,lat
0,Аналитик,fullDay,Алматы,700000.0,900000.0,KZT,76.945728,43.236392,146,76.945728,43.236392,76.945728,43.236392
1,Бизнес-аналитик,fullDay,Алматы,NaN,NaN,NaN,76.945728,43.236392,146,76.945728,43.236392,76.945728,43.236392
2,Аналитик данных (бренд аналитик),fullDay,Алматы,300000.0,NaN,KZT,76.945728,43.236392,146,76.945728,43.236392,76.945728,43.236392
3,Аналитик,fullDay,Алматы,200000.0,200000.0,KZT,76.945728,43.236392,146,76.945728,43.236392,76.945728,43.236392
4,Аналитик,fullDay,Алматы,NaN,NaN,NaN,76.945728,43.236392,146,76.945728,43.236392,76.945728,43.236392


In [15]:
# from urllib.request import urlopen
# import json
# with urlopen(link) as response:
#     counties = json.load(response)

In [138]:
fig = px.scatter_geo(df, lat='lat', lon='lon', color='total_vac', text='city', size='total_vac', width=1600, height=1600)
fig.show()

In [18]:
cities = pd.DataFrame()

cities['total_vac'] = (df
 .groupby('city')['name']
 .agg('count')
 )

cities['lat'] = (df
 .groupby('city')
 .agg({'lat': 'mean'})
 )

cities['lon'] = (df
 .groupby('city')
 .agg({'lon': 'mean'})
 )

cities['city'] = cities.index

cities.head()

,total_vac,lat,lon,city
city,,,,
Анапа,1,44.891491,37.337217,Анапа
Барнаул,4,53.348933,83.718417,Барнаул
Белгород,3,50.606482,36.592601,Белгород
Бийск,1,52.548723,85.234319,Бийск
Благовещенск (Амурская область),2,50.267470,127.538681,Благовещенск (Амурская область)


In [19]:
fig = px.scatter_geo(cities, lat='lat', lon='lon', text='city', color='total_vac', size='total_vac', width=1600, height=1600)
fig.show()

In [33]:
link = "https://raw.githubusercontent.com/timurkanaz/Russia_geojson_OSM/master/GeoJson's/Countries/Russia_regions.geojson"
from urllib.request import urlopen
import json
with urlopen(link) as response:
    counties = json.load(response)

In [34]:
for k in range(len(counties['features'])):
    counties['features'][k]['id'] = k

In [37]:
region_id_list = []
regions_list = []
for k in range(len(counties['features'])):
    region_id_list.append(counties['features'][k]['id'])
    regions_list.append(counties['features'][k]['properties']['region'])
df_regions = pd.DataFrame()
df_regions['region_id'] = region_id_list
df_regions['region_name'] = regions_list

In [38]:
df_regions

,region_id,region_name
0,0,Алтайский край
1,1,Амурская обл.
2,2,Архангельская обл.
3,3,Астраханская обл.
4,4,Белгородская обл.
...,...,...
80,80,Ямало-Ненецкий АО
81,81,Ярославская обл.
82,82,г. Москва
83,83,г. Санкт-Петербург
